In [1]:
!pip install pyfastx obonet protein-bert

  Using cached pyfastx-1.1.0-cp310-cp310-macosx_11_0_arm64.whl (720 kB)
  Using cached protein_bert-1.0.1-py3-none-any.whl (39 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.3 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached tensorflow_addons-0.20.0-cp310-cp310-macosx_11_0_arm64.whl (11.6 MB)
  Using cached pyfaidx-0.7.2.1-py3-none-any.whl (28 kB)
  Using cached lxml-4.9.2-cp310-cp310-macosx_11_0_arm64.whl
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: /nix/store/3i3wfmaz78mwzpm6h2cn6lskvpavbcsb-python3-3.10.10-env/bin/python3.10 -m pip install --upgrade pip


In [12]:
import pyfastx
import obonet
import pandas as pd

from tensorflow import keras
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, PretrainingModelGenerator, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

In [2]:
![ ! -f train_sequences.fasta ] && cp ../input/cafa-5-protein-function-prediction/Train/train_sequences.fasta .
fa = pyfastx.Fasta('train_sequences.fasta')
all_seqs = {s.name: s.seq for s in fa}

In [3]:
url = '../input/cafa-5-protein-function-prediction/Train/go-basic.obo'
graph = obonet.read_obo(url)
all_terms = list(graph.nodes())
all_labels = list(range(len(all_terms)))
term_to_label = {term: label for term, label in zip(all_terms, all_labels)}

In [4]:
train = pd.read_csv('../input/cafa-5-protein-function-prediction/Train/train_terms.tsv', sep='\t')

# Filter out terms with only 1 sample (since they cannot be splitted into train and validation sets)
train = train.groupby('term').filter(lambda x: x['EntryID'].count() >= 2)

train['text'] = train['EntryID'].map(lambda x: all_seqs[x])
train['label'] = train['term'].map(lambda x: term_to_label[x])

In [28]:
OUTPUT_TYPE = OutputType(False, 'categorical')
UNIQUE_LABELS = all_labels
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

train_set, valid_set = train_test_split(train, stratify = train['label'], test_size = 0.1, random_state = 0)

pretrained_model_generator, input_encoder = load_pretrained_model(
    local_model_dump_dir = '.'
)

 Local model dump file ./default.pkl doesn't exist. Will download ftp://ftp.cs.huji.ac.il/users/nadavb/protein_bert/epoch_92400_sample_23500000.pkl into .. Please approve or reject this (to exit and potentially call the function again with different parameters).
Do you approve downloadig the file into the specified directory? Please specify "Yes" or "No":yes
Downloaded file: ./epoch_92400_sample_23500000.pkl
Created: ./default.pkl


In [31]:
y = pretrained_model_generator.create_model(seq_len = 512).predict(input_encoder.encode_X(
    ['MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENARIQSKLSDLQKKKIDIDNKLLKEKQNLIKEEILERKKLEVLTKKQQKDEIEHQKKLKREIDAIKA\
    STQYITDVSISSYNNTIPETEPEYDLFISHASEDKEDFVRPLAETLQQLGVNVWYDEFTLKVGDSLRQKIDSGLRNSKYGTVVLSTDFIKKDWTNYELDGLVAREMNGHKMIL\
    PIWHKITKNDVLDYSPNLADKVALNTSVNSIEEIAHQLADVILNR'],
seq_len = 512))
dir(y[1])

1/1 [==============================] - 1s 671ms/step


['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [ ]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['text'], train_set['label'], valid_set['text'], valid_set['label'], \
        seq_len = 512, batch_size = 32, max_epochs_per_stage = 40, lr = 1e-04, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = 1e-05, callbacks = training_callbacks)
